In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
import numpy as np
import operator as op
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import io
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

In [ ]:
file_train = '/content/drive/My Drive/Analyse_des_sentiment/data/train.xml'
file_dev   = '/content/drive/My Drive/Analyse_des_sentiment/data/dev.xml'
file_test  = '/content/drive/My Drive/Analyse_des_sentiment/data/test.xml'

In [ ]:
def read_xml_to_dataframe(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        movie = comment.find('movie').text
        review_id = comment.find('review_id').text
        name = comment.find('name').text
        user_id = comment.find('user_id').text
        note = comment.find('note').text.replace(',', '.')
        commentaire = comment.find('commentaire').text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append({
            'movie': movie,
            'review_id': review_id,
            'name': name,
            'user_id': user_id,
            'note': note,
            'commentaire': commentaire
        })

    # Création d'un DataFrame
    df = pd.DataFrame(data)
    return df

In [ ]:
#fonction pour le fichier de test
def read_xml_to_dataframeTest(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        # Initialisation d'un dictionnaire pour chaque commentaire
        comment_data = {}

        # Extraction de chaque élément si disponible et ajout au dictionnaire
        movie = comment.find('movie')
        if movie is not None:
            comment_data['movie'] = movie.text

        review_id = comment.find('review_id')
        if review_id is not None:
            comment_data['review_id'] = review_id.text

        name = comment.find('name')
        if name is not None:
            comment_data['name'] = name.text

        user_id = comment.find('user_id')
        if user_id is not None:
            comment_data['user_id'] = user_id.text


        note = comment.find('note')
        if note is not None:
            comment_data['note'] = note.text.replace(',', '.')

        commentaire = comment.find('commentaire')
        if commentaire is not None:
            comment_data['commentaire'] = commentaire.text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append(comment_data)

    # Création d'un DataFrame
    df = pd.DataFrame(data)

    return df


In [ ]:
"""# Récupération des données dans des dataFrame"""

#création DataFrame
data_dev   = read_xml_to_dataframe(file_dev)
data_train = read_xml_to_dataframe(file_train)
data_test = read_xml_to_dataframeTest(file_test)


In [ ]:
import csv
import pandas as pd

file_path = '/content/drive/My Drive/processed_data/POLARITY-JEUXDEMOTS-FR.csv'
temp_file_path = '/content/drive/My Drive/processed_data/temp_lexicon.csv'


data = []

with open(file_path, 'r', encoding='latin1') as file:
    for line in file:
        if line.startswith('//'):  # Check for comment line
            continue

        # Split the line by semicolon, but not those inside quotes
        parts = line.strip().split(';')

        # Handle each part
        if len(parts) == 5:
            line_number, term, positive, neutral, negative = parts
            # Remove quotes from term if present
            term = term.strip('"')
            data.append([line_number, term, positive, neutral, negative])

# Convert the list of data into a DataFrame
lexicon_df = pd.DataFrame(data, columns=['line_number', 'term', 'positive', 'neutral', 'negative'])

# Create a dictionary mapping terms to their sentiment scores
sentiment_lexicon = {
    term: {
        'positive': int(row['positive']),
        'negative': int(row['negative']),
        'neutral': int(row['neutral'])
    }
    for term, row in lexicon_df.iterrows() if row['term'] not in ['terme', '']
}

# Display the DataFrame to verify
print(lexicon_df)



       line_number         term positive neutral negative
0                1            =       75      37       31
1                2           =)      236      47        0
2                3            >       22      21       21
3                4            -      381      49       74
4                5           -é       68     245        0
...            ...          ...      ...     ...      ...
642378      642883      Z>Zorro       98      61        0
642379      642884       ZZ Top      269      58       38
642380      642885          zzz      108     115      105
642381      642886   zzz>dormir        3       2        2
642382      642887  zzz>insecte        3       2        2

[642383 rows x 5 columns]


In [ ]:
def get_sentiment_score(document):
    positive_score = 0
    negative_score = 0
    neutral_score = 0

    words = document.split()  # Split into words
    for word in words:
        if word in sentiment_lexicon:
            sentiment = sentiment_lexicon[word]
            positive_score += sentiment['positive']
            negative_score += sentiment['negative']
            neutral_score += sentiment['neutral']

    total_score = positive_score - negative_score + neutral_score
    return total_score


In [ ]:

data_train = data_train.dropna(subset=['commentaire'])
data_dev = data_dev.dropna(subset=['commentaire'])
data_test = data_test.dropna(subset=['commentaire'])

In [ ]:
data_train['sentiment_score'] = data_train['commentaire'].apply(get_sentiment_score)
data_dev['sentiment_score'] = data_dev['commentaire'].apply(get_sentiment_score)
data_test['sentiment_score'] = data_test['commentaire'].apply(get_sentiment_score)
